In [1]:
from __future__ import print_function

import argparse
import contextlib
import fileinput
import gzip
import itertools
import os
import shutil
import subprocess
import six
import sys
import tempfile
import time
from collections import defaultdict
from distutils.spawn import find_executable

In [2]:
# basic functions

In [9]:
@contextlib.contextmanager
def file_transaction(*rollback_files):
    """
    Wrap file generation in a transaction, moving to output if finishes.
    """
    safe_names, orig_names = _flatten_plus_safe(rollback_files)
    # remove any half-finished transactions
    remove_files(safe_names)
    try:
        if len(safe_names) == 1:
            yield safe_names[0]
        else:
            yield tuple(safe_names)
    # failure -- delete any temporary files
    except:
        remove_files(safe_names)
        remove_tmpdirs(safe_names)
        raise
    # worked -- move the temporary files to permanent location
    else:
        for safe, orig in zip(safe_names, orig_names):
            if os.path.exists(safe):
                shutil.move(safe, orig)
        remove_tmpdirs(safe_names)


def remove_tmpdirs(fnames):
    for x in fnames:
        xdir = os.path.dirname(os.path.abspath(x))
        if xdir and os.path.exists(xdir):
            shutil.rmtree(xdir, ignore_errors=True)


def remove_files(fnames):
    for x in fnames:
        if x and os.path.exists(x):
            if os.path.isfile(x):
                os.remove(x)
            elif os.path.isdir(x):
                shutil.rmtree(x, ignore_errors=True)


def _flatten_plus_safe(rollback_files):
    """
    Flatten names of files and create temporary file names.
    """
    tx_files, orig_files = [], []
    for fnames in rollback_files:
        if isinstance(fnames, six.string_types):
            fnames = [fnames]
        for fname in fnames:
            basedir = safe_makedir(os.path.dirname(fname))
            tmpdir = safe_makedir(tempfile.mkdtemp(dir=basedir))
            tx_file = os.path.join(tmpdir, os.path.basename(fname))
            tx_files.append(tx_file)
            orig_files.append(fname)
    return tx_files, orig_files


def safe_makedir(dname):
    """
    Make a directory if it doesn't exist, handling concurrent race conditions.
    """
    if not dname:
        return dname
    num_tries = 0
    max_tries = 5
    while not os.path.exists(dname):
        try:
            os.makedirs(dname)
        except OSError:
            if num_tries > max_tries:
                raise
            num_tries += 1
            time.sleep(2)
    return dname


def file_exists(fnames):
    """
    Check if a file or files exist and are non-empty.

    parameters
        fnames : file path as string or paths as list; if list, all must exist

    returns
        boolean
    """
    if isinstance(fnames, six.string_types):
        fnames = [fnames]
    for f in fnames:
        if not os.path.exists(f) or os.path.getsize(f) == 0:
            return False
    return True


def check_dependencies(executables):
    exes = []
    for exe in executables:
        if not find_executable(exe):
            exes.append(exe)
    if len(exes) > 0:
        for exe in exes:
            print("`%s` not found in PATH." % exe)
        sys.exit(1)


def name_from_path(path):
    file, ext = os.path.splitext(os.path.basename(path))
    if ext == ".gz":
        file, ext = os.path.splitext(file)
    return file

def readfa(fh):
    for header, group in itertools.groupby(fh, lambda line: line[0] == '>'):
        if header:
            line = next(group)
            name = line[1:].strip()
        else:
            seq = ''.join(line.strip() for line in group)
            yield name, seq
            
def format_fasta_record(name, seq, wrap=100):
    """Fasta __str__ method.

    Convert fasta name and sequence into wrapped fasta format.

    Args:
        name (str): name of the record
        seq (str): sequence of the record
        wrap (int): length of sequence per line

    Returns:
        tuple: name, sequence

    >>> format_fasta_record("seq1", "ACTG")
    ">seq1\nACTG"
    """
    record = ">{name}\n".format(name=name)
    if wrap:
        for i in range(0, len(seq), wrap):
            record += seq[i:i+wrap] + "\n"
    else:
        record += seq + "\n"
    return record.strip()

In [91]:
def run_cd_hit(input_fa, output_fa, c=0.9, G=1, b=20, M=800,
    T=1, n=5, l=10, t=2, d=20, s=0.0, S=999999, aL=0.0, AL=99999999, aS=0.0,
    AS=99999999, A=0, uL=1.0, uS=1.0, U=99999999, g=1, sc=0, sf=0):
    """Run CD-HIT to cluster input FASTA.

    Args:
        input_fa (str): File path to fasta.
        output_fa (str): File path of output fasta.
        c (Optional[float]): sequence identity threshold, default 0.9
 	        this is the default cd-hit's "global sequence identity" calculated as:
 	        number of identical amino acids in alignment
            divided by the full length of the shorter sequence
        G (Optional[int]): use global sequence identity, default 1
 	        if set to 0, then use local sequence identity, calculated as :
            number of identical amino acids in alignment
 	        divided by the length of the alignment
 	        NOTE!!! don't use -G 0 unless you use alignment coverage controls
 	        see options -aL, -AL, -aS, -AS
        b (Optional[int]): band_width of alignment, default 20
        M (Optional[int]): memory limit (in MB) for the program, default 800; 0 for unlimited
        T (Optional[int]): number of threads, default 1; with 0, all CPUs will be used
        n (Optional[int]): word_length, default 5, see user's guide for choosing it
        l (Optional[int]): length of throw_away_sequences, default 10
        t (Optional[int]): tolerance for redundance, default 2
        d (Optional[int]): length of description in .clstr file, default 20
 	        if set to 0, it takes the fasta defline and stops at first space
        s (Optional[float]): length difference cutoff, default 0.0
 	        if set to 0.9, the shorter sequences need to be
            at least 90% length of the representative of the cluster
        S (Optional[int]): length difference cutoff in amino acid, default 999999
 	        if set to 60, the length difference between the shorter sequences
 	        and the representative of the cluster can not be bigger than 60
        aL (Optional[float]): alignment coverage for the longer sequence, default 0.0
 	        if set to 0.9, the alignment must covers 90% of the sequence
        AL (Optional[int]): alignment coverage control for the longer sequence, default 99999999
 	        if set to 60, and the length of the sequence is 400,
 	        then the alignment must be >= 340 (400-60) residues
        aS (Optional[float]): alignment coverage for the shorter sequence, default 0.0
        	if set to 0.9, the alignment must covers 90% of the sequence
        AS (Optional[int]): alignment coverage control for the shorter sequence, default 99999999
        	if set to 60, and the length of the sequence is 400,
        	then the alignment must be >= 340 (400-60) residues
        A (Optional[int]): minimal alignment coverage control for the both sequences, default 0
        	alignment must cover >= this value for both sequences
        uL (Optional[float]): maximum unmatched percentage for the longer sequence, default 1.0
        	if set to 0.1, the unmatched region (excluding leading and tailing gaps)
        	must not be more than 10% of the sequence
        uS (Optional[float]): maximum unmatched percentage for the shorter sequence, default 1.0
        	if set to 0.1, the unmatched region (excluding leading and tailing gaps)
        	must not be more than 10% of the sequence
        U (Optional[int]): maximum unmatched length, default 99999999
        	if set to 10, the unmatched region (excluding leading and tailing gaps)
        	must not be more than 10 bases
        g (Optional[int]): 1 or 0, default 1
        	when 0 a sequence is clustered to the first
        	cluster that meet the threshold (fast cluster). If set to 1, the program
        	will cluster it into the most similar cluster that meet the threshold
        	(accurate but slow mode)
        	but either 1 or 0 won't change the representatives of final clusters
        sc (Optional[int]): sort clusters by size (number of sequences), default 0, output clusters by decreasing length
        	if set to 1, output clusters by decreasing size
        sf (Optional[int]): sort fasta/fastq by cluster size (number of sequences), default 0, no sorting
        	if set to 1, output sequences by decreasing cluster size

    Returns:
        list, [file path of output fasta, file path of output cluster definitions]

    """
    output_clstr = "{fa}.clstr".format(fa=output_fa)
    output_files = [output_fa, output_clstr]
    if file_exists(output_files):
        return output_files

    print("Running CD-HIT on {fa}".format(fa=input_fa), file=sys.stderr)

    contig_name_map = {}
    tmp_fasta = "{fa}.rename.tmp".format(fa=input_fa)
    with open(input_fa) as f_in, open(tmp_fasta, "w") as f_out:
        for i, (name, seq) in enumerate(readfa(f_in), start=1):
            contig_name_map["%d" % i] = name
            print(format_fasta_record(i, seq), file=f_out)

    with file_transaction(output_files) as tx_out_files:
        cmd = ("cd-hit -i {input_fasta} -o {output_fasta} -c {c} "
                "-G {G} -b {b} -M {M} -T {T} -n {n} -l {l} -t {t} "
                "-d {d} -s {s} -S {S} -aL {aL} -AL {AL} -aS {aS} "
                "-AS {AS} -A {A} -uL {uL} -uS {uS} -U {U} "
                "-p 1 -g {g} -sc {sc} -sf {sf}").format(input_fasta=tmp_fasta,
                                                        output_fasta=tx_out_files[0],
                                                        **locals())
        subprocess.check_call(cmd, shell=True)
        # copy the clstr output to its temp file location; let file_transaction clean up
        shutil.copyfile("{fa}.clstr".format(fa=tx_out_files[0]), tx_out_files[1])

        # edit the output files in place back to their original names
        # changes the format of the cluster file

        # update change the contig names in the cluster file back to original
        for line in fileinput.input(tx_out_files[0], inplace=True):
            line = line.strip()
            if line.startswith(">"):
                name = contig_name_map[line.strip(">")]
                print(">{name}".format(name=name))
            else:
                print(line)

        # change the contig names in the cluster file
        for line in fileinput.input(tx_out_files[1], inplace=True):
            line = line.strip()
            if not line.startswith(">"):
                # changes:
                # 1	382aa, >6... at 1:382:1:382/100.00%
                # to just the original contig name
                if "*" in line:
                    print('{}*'.format(contig_name_map[line.partition(">")[-1].partition("...")[0]]))
                else:
                    print(contig_name_map[line.partition(">")[-1].partition("...")[0]])
            else:
                # this is the cluster ID
                print(line)

    if file_exists(tmp_fasta):
        os.remove(tmp_fasta)

    return output_files

In [92]:
input_fa = "/mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/AG-919-G14_AG-910-A04.faa"
output_fa = "./outputs/AG-919-G14_AG-910-A04_clusters.faa"

outs = run_cd_hit(input_fa, output_fa)

Running CD-HIT on /mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/AG-919-G14_AG-910-A04.faa


In [93]:
outs

['./outputs/AG-919-G14_AG-910-A04_clusters.faa',
 './outputs/AG-919-G14_AG-910-A04_clusters.faa.clstr']

In [94]:
!head {outs[1]}

>Cluster 0
AG-919-G14_00862 hypothetical protein
AG-910-A04_00192 hypothetical protein*
>Cluster 1
AG-919-G14_01049 Ferredoxin-dependent glutamate synthase 2*
AG-910-A04_00156 Ferredoxin-dependent glutamate synthase 2
>Cluster 2
AG-919-G14_00914 DNA-directed RNA polymerase subunit beta'*
>Cluster 3
AG-919-G14_00442 DNA polymerase III subunit alpha*


In [95]:
!head {outs[0]}

>AG-919-G14_00001 Trk system potassium uptake protein TrkA
MKVIIVGAHGEARQLINRISTGWNISVIDMDQEKLRNFNPNRQIEKYQGDGTSTLVLKKAGIENASALVTLTNDDEVNLEILKIAKENNIFRLSSIVNED
KNSQQYKDLDVEVVDPDVLIGRRLEHILEPRRVVSQAFAGGRAEAIELEINSDSPARGKKLKDIGSDFFIVGALLRKGNVVIPHGDTELETGDLVTIVLQ
SGAFSNVINLFSGSESRFPLEFGKNVAVFLKNEDELKNLSEAEYFIRNTKADKLEILTSGDIFPDQKEDQTDTYKAIMKDQDFELYNVQKSLLKEIETNK
DSFSIGTVLIPFDEKEITKSKLKTYINFSNKNNIPLLFSRSSFPYKKIGILVSDDFSDNSPVNIAFDLASTLSADVTALNISQPKFLQPEHASLSNKNTE
RIQDLALSNEVQCEIVNDEGNEAKVFSSFTDKIDLSIVSQTSQSNWQGKKIAEFILKNAKSSVLYIPN
>AG-919-G14_00002 Na(+)/H(+)-K(+) antiporter GerN
MENFDALSLVIISLGAFLLPLIAQRISIPSIVLEIAYGILVGPVLGIVKISEFISGLAIFGFMLLMFLSGFEIELDTFREKGLKTLSIPLAIYIATVATS
FYLIITLDYPMFLALVLCTTSVDIVITVLRSDDTIKTNYGQSLFMSALIADIFTLIGVTVFASLQHSEGFSISNLNVILYFLIVILILRIIRRVAWWNPQ
IFSRMFDGNDPEEMGIRSSIALMLTLVGVAVLFDIEYILGAFLAGTLFAFTFPNRGSLENSLKGFSYGFLIPIFFINIGLNYDIEVFKNTEFYIQVLSLL


In [96]:
clstr = "./outputs/AG-919-G14_AG-910-A04_clusters.faa.clstr"

In [101]:
def cluster_map(clstr):
    cluster_map = defaultdict(list)
    with open(clstr) as fh:
        for cluster_start, group in itertools.groupby(fh, lambda l: l[0] == '>'):
            members = []
            if not cluster_start: 
                for line in group:
                    if "*" in line: 
                        rep_seq = line.strip().replace("*", "")
                    else:
                        members.append(line.strip())
            if len(members) > 0:
                cluster_map[rep_seq] = members
    return cluster_map

In [102]:
len(cluster_map)

472

In [84]:
fanames = []

for name, seq in readfa(open(output_fa)):
    fanames.append(name)

In [85]:
len(set(list(cluster_map.keys())).intersection(fanames))

472

In [1]:
fasta="/mnt/scgc/scgc_nfs/lab/julia/notebooks/simons-viruscope/notebooks/outputs/AG-919-G14_AG-910-A04_clusters.faa"
clstr="/mnt/scgc/scgc_nfs/lab/julia/notebooks/simons-viruscope/notebooks/outputs/AG-919-G14_AG-910-A04_clusters.faa.clstr"
out_file="/mnt/scgc/scgc_nfs/lab/julia/notebooks/simons-viruscope/notebooks/outputs/AG-919-G14_AG-910-A04_mica_blast.out"
db = "/mnt/scgc/simon/databases/mica/nr-20150620-mica"
num_alignments=10
evalue=0.001
threads=60
fields = ["qseqid", "sseqid", "pident", "length", "mismatch",
                  "gapopen", "qstart", "qend", "sstart", "send", "evalue",
                  "bitscore", "sallseqid", "score", "nident", "positive",
                  "gaps", "ppos", "qframe", "sframe", "qseq", "sseq", "qlen",
                  "slen", "salltitles"]


cmd = ("mica-search --p='{threads}' --blastp 'blastp' {db} {query} "
                   "--blast-args -outfmt '\"6 {fields}\"' "
                   "-num_alignments {alignments} -evalue {evalue} -out {out}").format(db=db,
                                                      query=fasta,
                                                      fields=" ".join(fields),
                                                      threads=threads,
                                                      alignments=num_alignments,
                                                      evalue=evalue,
                                                      out=out_file)

In [18]:
fasta='/mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/uniprot-test-set.faa'
clstr="/mnt/scgc/scgc_nfs/lab/julia/notebooks/simons-viruscope/notebooks/outputs/AG-919-G14_AG-910-A04_clusters.faa.clstr"
out_file="/mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/notebooks/outputs/uniprot-test-set_mica_blast.out"
db = "/mnt/scgc/simon/databases/mica/nr-20150620-mica"
num_alignments=10
evalue=0.001
threads=20
fields = ["qseqid", "sseqid", "pident", "length", "mismatch",
                  "gapopen", "qstart", "qend", "sstart", "send", "evalue",
                  "bitscore", "sallseqid", "score", "nident", "positive",
                  "gaps", "ppos", "qframe", "sframe", "qseq", "sseq", "qlen",
                  "slen", "salltitles"]


cmd = ("mica-search --p='{threads}' --blastp 'blastp' {db} {query} "
                   "--blast-args -outfmt '6 {fields}' "
                   "-num_alignments {alignments} -evalue {evalue} -out {out}").format(db=db,
                                                      query=fasta,
                                                      fields=" ".join(fields),
                                                      threads=threads,
                                                      alignments=num_alignments,
                                                      evalue=evalue,
                                                      out=out_file)

In [19]:
print(cmd)

mica-search --p='20' --blastp 'blastp' /mnt/scgc/simon/databases/mica/nr-20150620-mica /mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/uniprot-test-set.faa --blast-args -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore sallseqid score nident positive gaps ppos qframe sframe qseq sseq qlen slen salltitles' -num_alignments 10 -evalue 0.001 -out /mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/notebooks/outputs/uniprot-test-set_mica_blast.out


In [10]:
pwd

'/mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/notebooks'

In [20]:
print("echo '{cmd}' | qsub -N mica_test -q route -V -l walltime=1:00:00,ncpus=20,mem=150G -j oe -o /home/julia/out/171219_mica.out".format(cmd = cmd))

echo 'mica-search --p='20' --blastp 'blastp' /mnt/scgc/simon/databases/mica/nr-20150620-mica /mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/viruscope/data/uniprot-test-set.faa --blast-args -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore sallseqid score nident positive gaps ppos qframe sframe qseq sseq qlen slen salltitles' -num_alignments 10 -evalue 0.001 -out /mnt/scgc_nfs/lab/julia/notebooks/simons-viruscope/notebooks/outputs/uniprot-test-set_mica_blast.out' | qsub -N mica_test -q route -V -l walltime=1:00:00,ncpus=20,mem=150G -j oe -o /home/julia/out/171219_mica.out


blastp -db /dev/shm/mica-fine-search-db552529519/blastdb-fine -dbsize 24387073819 -num_threads 60 -outfmt 6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore sallseqid score nident positive gaps ppos qframe sframe qseq sseq qlen slen salltitles -num_alignments 10 -evalue 0.001 -out AAA164-A21_vs_nr_mica_blastp.out

blastp -db /dev/shm/mica-fine-search-db027200658/blastdb-fine -dbsize 24387073819 -num_threads 20 -outfmt 6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore sallseqid score nident positive gaps ppos qframe sframe qseq sseq qlen slen salltitles -num_alignments 10 -evalue 0.001 -out /mnt/scgc/scgc_nfs/lab/julia/notebooks/simons-viruscope/notebooks/outputs/uniprot-test-set_mica_blast.out

In [126]:
!grep -c ">" {output_fa}

1158


In [129]:
print('diamond makedb --in {output_fa} -d {output_fa}.dmnd -p {threads}'.format(output_fa = output_fa, threads=40))

diamond makedb --in ./outputs/AG-919-G14_AG-910-A04_clusters.faa -d ./outputs/AG-919-G14_AG-910-A04_clusters.faa.dmnd -p 40


In [132]:
def make_diamond_db(fasta, db, threads=1, verbose=False):
    out_file = db + ".dmnd"
    if file_exists(out_file):
        return db

    if verbose:
        print("Creating DIAMOND database for", fasta, file=sys.stderr)

    with file_transaction(out_file) as tx_out_file:
        cmd = ("diamond makedb --in {fasta} -d {db} "
               "-p {threads}").format(fasta=fasta,
                                      db=tx_out_file,
                                      threads=threads)
        subprocess.check_call(cmd, shell=True)
    return db


def diamond_blastx(fasta, out_file, db, threads=1, verbose=False):
    if file_exists(out_file):
        return out_file

    if verbose:
        print("Running DIAMOND BLASTX on %s across %s" %
              (os.path.basename(fasta), os.path.basename(db)),
              file=sys.stderr)

    with file_transaction(out_file) as tx_out_file:
        cmd = ("diamond blastx -d {db} -q {fasta} "
               "-a {out} -p {threads} -t {tmpdir}").format(db=db,
                                                           fasta=fasta,
                                                           out=tx_out_file,
                                                           threads=threads,
                                                           tmpdir=tempfile.gettempdir())
        subprocess.check_call(cmd, shell=True)
    return out_file


def diamond_view(daa, out_file, verbose=False):
    if file_exists(out_file):
        return out_file

    if verbose:
        print("Converting DIAMOND database %s to tabular (%s)" %
              (os.path.basename(daa), os.path.basename(out_file)),
              file=sys.stderr)

    with file_transaction(out_file) as tx_out_file:
        nongz = tx_out_file.rpartition(".")[0]
        subprocess.check_call(["diamond", "view", "-a", daa, "-o", nongz])
        subprocess.check_call(["gzip", nongz])
    return out_file

def _sam_to_bam(sam, idx, out_file, threads=8):
    cmd = ("samtools view -@ {t} -bSht {index} {sam} 2> /dev/null "
           "| samtools sort -@ {t} -m 2G - > {bam} 2> /dev/null"
          ).format(t=threads,
                   index=idx,
                   sam=sam,
                   bam=out_file.rpartition(".")[0] + ".bam")
    subprocess.check_call(cmd, shell=True)
    return out_file

In [133]:
daa_out = "./outputs/AG-919-G14_AG-910-A04_vs_POV.daa"
sam_out = "./outputs/AG-919-G14_AG-910-A04_vs_POV.sam"

In [135]:
import pysam

def read_overlap_pctid(l, pctid, min_len, overlap=0):
    ''' looks at pysam.AligmentFile line and determines if mapping matches input parms'''
    real_len = l.infer_query_length()
    aln_len = l.query_alignment_length
    mismatch = l.get_tag("NM")

    aln_overlap = (aln_len / real_len) * 100
    aln_pctid = ((aln_len - mismatch) / aln_len) * 100
    if aln_overlap >= overlap and aln_pctid >= pctid and aln_len >= min_len:
        return True
    else:
        return False

In [ ]:
!samtools view -@ 5 -bSht

In [137]:
umcount = 0
good_ct = 0
bad_ct = 0

with pysam.AlignmentFile(sam_out, "rb", check_sq=False) as ih:
    good = 0
    good_bp = 0
    total = 0
    for i, l in enumerate(ih):
        if l.is_duplicate:
            continue

        total += 1
        print(l.infer_query_length())
        if l.is_unmapped():
            umcount += 1
            continue
            
        if read_overlap_pctid(l, 95, 100):
            good_ct += 1
        else:
            bad_ct += 1

NotImplementedError: can not iterate over samfile without header